# Comparison of results in initial and latent space

[1. Optimize LSH](#1-optimize-lsh)

[2. Optimize Hypercube](#2-optimize-hypercube)

[3. Optimize GNNS](#3-optimize-gnns)

[4. Optimize MRNG](#4-optimize-mrng)

[5. Optimize NSG](#5-optimize-nsg)

[6. Grid Search](#6-grid-search)

[7. Results](#7-results)

+ [7. a. Optimization Results](#7-a-optimization-results)

+ [7. b. Grid Search Results](#7-b-grid-search-results)

[8. Conclusions](#8-conclusions)

# Import libraries

In [1]:
import os

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder
from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

from params import lsh_test, hypercube_test, kmeans_test, gnn_test, mrng_test, nsg_test, get_aaf

2024-01-01 22:51:15.193790: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 22:51:15.234563: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 22:51:15.235110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 22:51:16.027163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'
    decoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_dataset.dat'
    decoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [3]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

313/313 [==============================] - 1s 2ms/step


# 1. Optimize LSH

To skip logs, click [here](#visualize-lsh-study-results).

In [4]:
# to be copied from optimize_lsh.ipynb (3 cells)

## Visualize LSH study results

In [5]:
# to be copied from optimize_lsh.ipynb (5 cells)

# 2. Optimize Hypercube

To skip logs, click [here](#visualize-hypercube-study-results).

In [4]:
def objective_hypercube(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'M': trial.suggest_int('M', 10, 100),
                  'N': 10,
                  'window': trial.suggest_float('window_size', 0.01, 1)}
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]
    
    average_time, aaf_latent = hypercube_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, probes = 1000)

    return aaf_latent.value, average_time.value

In [5]:
%%time
hypercube_study = optuna.create_study(study_name='hypercube', directions=['minimize', 'minimize'])
hypercube_study.optimize(objective_hypercube, n_trials=50, n_jobs=-1)
print("-----------------------------------------------------")

trials = sorted(hypercube_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2024-01-01 22:51:52,314] A new study created in memory with name: hypercube


Trial parameters: {'k': 8, 'M': 13, 'N': 10, 'window': 0.3380375279630569}
Trial parameters: {'k': 8, 'M': 88, 'N': 10, 'window': 0.9834470456417957}
Trial parameters: {'k': 4, 'M': 44, 'N': 10, 'window': 0.5824115086662383}
Trial parameters: {'k': 6, 'M': 87, 'N': 10, 'window': 0.3245806943132081}
Trial parameters: {'k': 3, 'M': 81, 'N': 10, 'window': 0.03216886526933036}
Trial parameters: {'k': 8, 'M': 54, 'N': 10, 'window': 0.6578930026401334}
Trial parameters: {'k': 6, 'M': 80, 'N': 10, 'window': 0.4019180428790731}
Trial parameters: {'k': 10, 'M': 45, 'N': 10, 'window': 0.8639177811257001}
Trial parameters: {'k': 4, 'M': 11, 'N': 10, 'window': 0.9225102974036616}
Trial parameters: {'k': 7, 'M': 96, 'N': 10, 'window': 0.5267289602825778}
Trial parameters: {'k': 2, 'M': 58, 'N': 10, 'window': 0.5762007443870133}
Trial parameters: {'k': 7, 'M': 78, 'N': 10, 'window': 0.22773767524138688}


[I 2024-01-01 22:51:55,631] Trial 2 finished with values: [1.4765510732634153, 0.004268780000000001] and parameters: {'model': 'model_conv_46.keras', 'k': 4, 'M': 44, 'window_size': 0.5824115086662383}. 


Trial parameters: {'k': 5, 'M': 26, 'N': 10, 'window': 0.7200461552122244}


[I 2024-01-01 22:51:56,453] Trial 6 finished with values: [1.4726633914395573, 0.008987369999999998] and parameters: {'model': 'model_conv_46.keras', 'k': 6, 'M': 80, 'window_size': 0.4019180428790731}. 


...

Trial parameters: {'k': 7, 'M': 90, 'N': 10, 'window': 0.6639455657654231}


[I 2024-01-01 22:52:12,910] Trial 40 finished with values: [1.537731761600282, 0.009671249999999998] and parameters: {'model': 'model_dense_26.keras', 'k': 6, 'M': 51, 'window_size': 0.9224224117770076}. 
[I 2024-01-01 22:52:13,057] Trial 36 finished with values: [1.36311242540067, 0.037427319999999986] and parameters: {'model': 'model_dense_43.keras', 'k': 10, 'M': 45, 'window_size': 0.25225313470930877}. 
[I 2024-01-01 22:52:13,114] Trial 39 finished with values: [1.2792391314621974, 0.00505505] and parameters: {'model': 'model_conv_19.keras', 'k': 2, 'M': 29, 'window_size': 0.10699092161777747}. 


Trial parameters: {'k': 3, 'M': 67, 'N': 10, 'window': 0.420092695618134}


[I 2024-01-01 22:52:13,261] Trial 42 finished with values: [1.347234340491104, 0.009066759999999998] and parameters: {'model': 'model_conv_12.keras', 'k': 5, 'M': 86, 'window_size': 0.7019105283412654}. 
[I 2024-01-01 22:52:13,697] Trial 44 finished with values: [1.3425234373996997, 0.005239660000000002] and parameters: {'model': 'model_conv_12.keras', 'k': 3, 'M': 49, 'window_size': 0.4439924333912712}. 
[I 2024-01-01 22:52:14,091] Trial 41 finished with values: [1.2688370713025698, 0.016811390000000006] and parameters: {'model': 'model_conv_19.keras', 'k': 8, 'M': 76, 'window_size': 0.6343889080305924}. 
[I 2024-01-01 22:52:14,424] Trial 45 finished with values: [1.546704079525005, 0.008229840000000006] and parameters: {'model': 'model_dense_26.keras', 'k': 7, 'M': 61, 'window_size': 0.8305576749098839}. 
[I 2024-01-01 22:52:14,695] Trial 47 finished with values: [1.459378762887225, 0.008880940000000002] and parameters: {'model': 'model_conv_46.keras', 'k': 8, 'M': 74, 'window_size':

-----------------------------------------------------
Trial no. 3
 Values = [1.2176417254593472, 0.013224399999999999]
 Params = {'model': 'model_dense_1.keras', 'k': 6, 'M': 87, 'window_size': 0.3245806943132081}
Trial no. 24
 Values = [1.2204518124145651, 0.009401259999999998]
 Params = {'model': 'model_dense_1.keras', 'k': 6, 'M': 44, 'window_size': 0.21347906548983225}
Trial no. 8
 Values = [1.2205153118487573, 0.00812019]
 Params = {'model': 'model_dense_1.keras', 'k': 4, 'M': 11, 'window_size': 0.9225102974036616}
Trial no. 25
 Values = [1.221396937309805, 0.006781179999999998]
 Params = {'model': 'model_dense_1.keras', 'k': 5, 'M': 55, 'window_size': 0.792302820217333}
Trial no. 43
 Values = [1.2226471505237209, 0.0047069500000000005]
 Params = {'model': 'model_dense_1.keras', 'k': 6, 'M': 100, 'window_size': 0.9578479683856898}
Trial no. 10
 Values = [1.2749578092315483, 0.0035873899999999985]
 Params = {'model': 'model_conv_19.keras', 'k': 2, 'M': 58, 'window_size': 0.57620074

In [6]:
df = hypercube_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_M,params_k,params_model,params_window_size,system_attrs_nsga2:generation,state
0,3,1.217642,0.013224,2024-01-01 22:51:52.320352,2024-01-01 22:51:59.651566,0 days 00:00:07.331214,87,6,model_dense_1.keras,0.324581,0,COMPLETE
1,34,1.219278,0.016952,2024-01-01 22:52:04.656224,2024-01-01 22:52:12.402686,0 days 00:00:07.746462,51,9,model_dense_1.keras,0.545178,0,COMPLETE
2,24,1.220452,0.009401,2024-01-01 22:52:00.880399,2024-01-01 22:52:08.131551,0 days 00:00:07.251152,44,6,model_dense_1.keras,0.213479,0,COMPLETE
3,8,1.220515,0.008120,2024-01-01 22:51:52.325699,2024-01-01 22:51:57.597702,0 days 00:00:05.272003,11,4,model_dense_1.keras,0.922510,0,COMPLETE
4,25,1.221397,0.006781,2024-01-01 22:52:00.888268,2024-01-01 22:52:08.360245,0 days 00:00:07.471977,55,5,model_dense_1.keras,0.792303,0,COMPLETE
5,38,1.221421,0.013871,2024-01-01 22:52:07.578207,2024-01-01 22:52:14.723800,0 days 00:00:07.145593,74,8,model_dense_1.keras,0.863974,0,COMPLETE
6,43,1.222647,0.004707,2024-01-01 22:52:08.777255,2024-01-01 22:52:15.186241,0 days 00:00:06.408986,100,6,model_dense_1.keras,0.957848,0,COMPLETE
7,28,1.224449,0.006942,2024-01-01 22:52:01.992199,2024-01-01 22:52:08.427964,0 days 00:00:06.435765,94,2,model_dense_1.keras,0.217929,0,COMPLETE
8,0,1.229488,0.017162,2024-01-01 22:51:52.317440,2024-01-01 22:52:00.345190,0 days 00:00:08.027750,13,8,model_dense_1.keras,0.338038,0,COMPLETE
9,9,1.260619,0.009793,2024-01-01 22:51:52.326604,2024-01-01 22:51:58.040870,0 days 00:00:05.714266,96,7,model_conv_19.keras,0.526729,0,COMPLETE


## Visualize Hypercube study results

In [7]:
plot_pareto_front(hypercube_study, target_names=['aaf', 'average_time'])

In [8]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[0], target_name = 'aaf')

In [9]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

In [10]:
plot_slice(hypercube_study, target = lambda t: t.values[0], target_name = 'aaf')

In [11]:
plot_slice(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

# 3. Optimize GNNS

To skip logs, click [here](#visualize-gnns-study-results).

In [30]:
def objective_gnns(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'k': trial.suggest_int('k', 40, 100)}
    param_dict.update({'E': trial.suggest_int('E', 40, param_dict['k'])})
    param_dict.update({'R': trial.suggest_int('R', 1, 10)})

    print("Trial params", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]

    average_time, aaf_latent = gnn_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, N=5, int_data=0)

    return aaf_latent.value, average_time.value

In [31]:
%%time
gnns_study = optuna.create_study(study_name='gnns', directions=['minimize', 'minimize'])
gnns_study.optimize(objective_gnns, n_trials=100, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(gnns_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2024-01-01 09:28:23,339] A new study created in memory with name: gnns


Trial params {'k': 86, 'E': 62, 'R': 8}
Trial params {'k': 92, 'E': 80, 'R': 10}
Trial params {'k': 98, 'E': 89, 'R': 3}
Trial params {'k': 43, 'E': 41, 'R': 10}


[I 2024-01-01 09:28:44,161] Trial 3 finished with values: [1.063268300989251, 0.005249319999999999] and parameters: {'model': 'model_dense_1.keras', 'k': 43, 'E': 41, 'R': 10}. 


Trial params {'k': 50, 'E': 42, 'R': 4}


[I 2024-01-01 09:28:44,744] Trial 1 finished with values: [1.0604634235668202, 0.009724120000000003] and parameters: {'model': 'model_dense_1.keras', 'k': 92, 'E': 80, 'R': 10}. 


...

Trial params {'k': 52, 'E': 50, 'R': 8}


[I 2024-01-01 10:00:46,657] Trial 98 finished with values: [1.0182138872078343, 0.006596340000000002] and parameters: {'model': 'model_conv_46.keras', 'k': 52, 'E': 50, 'R': 8}. 


Trial params {'k': 85, 'E': 79, 'R': 3}


[I 2024-01-01 10:00:54,404] Trial 97 finished with values: [1.0819645608497654, 0.0024116000000000003] and parameters: {'model': 'model_conv_19.keras', 'k': 49, 'E': 41, 'R': 3}. 
[I 2024-01-01 10:01:07,340] Trial 96 finished with values: [1.0242296574429919, 0.006094139999999997] and parameters: {'model': 'model_dense_26.keras', 'k': 85, 'E': 56, 'R': 5}. 
[I 2024-01-01 10:01:36,394] Trial 99 finished with values: [1.0408611629448543, 0.00209204] and parameters: {'model': 'model_conv_46.keras', 'k': 85, 'E': 79, 'R': 3}. 
[I 2024-01-01 10:03:14,976] Trial 80 finished with values: [1.0362188638603054, 0.006498760000000001] and parameters: {'model': 'model_dense_26.keras', 'k': 44, 'E': 42, 'R': 6}. 


-------------------- Best trials --------------------
Trial no. 58
 Values = [1.0114525841006923, 0.005807409999999998]
 Params = {'model': 'model_conv_46.keras', 'k': 92, 'E': 44, 'R': 9}
Trial no. 91
 Values = [1.0154519074349952, 0.00561651]
 Params = {'model': 'model_conv_12.keras', 'k': 86, 'E': 74, 'R': 6}
Trial no. 52
 Values = [1.0178442361059128, 0.005543559999999999]
 Params = {'model': 'model_conv_46.keras', 'k': 82, 'E': 47, 'R': 8}
Trial no. 33
 Values = [1.0253311720128528, 0.004564919999999998]
 Params = {'model': 'model_conv_12.keras', 'k': 86, 'E': 52, 'R': 6}
Trial no. 43
 Values = [1.028052116147886, 0.004392570000000001]
 Params = {'model': 'model_dense_26.keras', 'k': 85, 'E': 60, 'R': 5}
Trial no. 86
 Values = [1.0312514292725456, 0.0040866900000000005]
 Params = {'model': 'model_conv_46.keras', 'k': 40, 'E': 40, 'R': 6}
Trial no. 99
 Values = [1.0408611629448543, 0.00209204]
 Params = {'model': 'model_conv_46.keras', 'k': 85, 'E': 79, 'R': 3}
Trial no. 20
 Values

In [32]:
df = gnns_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_E,params_R,params_k,params_model,system_attrs_nsga2:generation,state
0,58,1.011453,0.005807,2024-01-01 09:45:34.059641,2024-01-01 09:46:17.792735,0 days 00:00:43.733094,44,9,92,model_conv_46.keras,1,COMPLETE
1,95,1.012020,0.006708,2024-01-01 09:57:58.265136,2024-01-01 10:00:01.378260,0 days 00:02:03.113124,59,8,86,model_conv_12.keras,1,COMPLETE
2,37,1.012940,0.008194,2024-01-01 09:37:10.934639,2024-01-01 09:38:11.059821,0 days 00:01:00.125182,87,7,96,model_conv_19.keras,0,COMPLETE
3,39,1.014485,0.008174,2024-01-01 09:37:33.882058,2024-01-01 09:39:22.805662,0 days 00:01:48.923604,69,9,70,model_dense_26.keras,0,COMPLETE
4,91,1.015452,0.005617,2024-01-01 09:56:51.750319,2024-01-01 09:57:58.261546,0 days 00:01:06.511227,74,6,86,model_conv_12.keras,1,COMPLETE
5,51,1.015664,0.008978,2024-01-01 09:42:36.416296,2024-01-01 09:45:07.019240,0 days 00:02:30.602944,71,5,91,model_conv_46.keras,0,COMPLETE
6,19,1.016073,0.007701,2024-01-01 09:32:53.888471,2024-01-01 09:34:08.998349,0 days 00:01:15.109878,58,9,66,model_conv_46.keras,0,COMPLETE
7,92,1.016301,0.006552,2024-01-01 09:57:13.371978,2024-01-01 09:57:54.581561,0 days 00:00:41.209583,64,7,64,model_conv_19.keras,1,COMPLETE
8,81,1.016650,0.012238,2024-01-01 09:53:24.308548,2024-01-01 09:54:58.068572,0 days 00:01:33.760024,63,9,69,model_dense_26.keras,1,COMPLETE
9,84,1.017505,0.006617,2024-01-01 09:53:55.716476,2024-01-01 09:54:48.458947,0 days 00:00:52.742471,61,8,80,model_conv_46.keras,1,COMPLETE


## Visualize GNNS study results

In [33]:
plot_pareto_front(gnns_study, target_names=['aaf', 'average_time'])

In [34]:
plot_optimization_history(gnns_study, target = lambda t: t.values[0], target_name = 'aaf')

In [35]:
plot_optimization_history(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

In [36]:
plot_slice(gnns_study, target = lambda t: t.values[0], target_name = 'aaf')

In [37]:
plot_slice(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

# 4. Optimize MRNG

To skip logs, click [here](#visualize-mrng-study-results).

In [38]:
def objective_mrng(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'l': trial.suggest_int('l', 1, 1000)}
    param_dict.update({'N': trial.suggest_int('N', 1, param_dict['l'])})
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]

    average_time, aaf_latent = mrng_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, int_data=0)

    return aaf_latent.value, average_time.value

In [39]:
%%time
mrng_study = optuna.create_study(study_name='mrng', directions=['minimize', 'minimize'])
mrng_study.optimize(objective_mrng, n_trials=50, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(mrng_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2024-01-01 10:03:15,278] A new study created in memory with name: mrng


Trial parameters: {'l': 124, 'N': 1}
Trial parameters: {'l': 540, 'N': 439}
Trial parameters: {'l': 442, 'N': 188}
Trial parameters: {'l': 162, 'N': 111}


[I 2024-01-01 10:20:22,600] Trial 1 finished with values: [1.1959783433738573, 0.0031169300000000004] and parameters: {'model': 'model_conv_46.keras', 'l': 540, 'N': 439}. 


Trial parameters: {'l': 910, 'N': 754}


[I 2024-01-01 10:23:05,390] Trial 0 finished with values: [1.3627979844757485, 0.002928019999999999] and parameters: {'model': 'model_dense_26.keras', 'l': 124, 'N': 1}. 


...

Trial parameters: {'l': 763, 'N': 343}


[I 2024-01-01 14:34:49,391] Trial 45 finished with values: [1.1472216027599742, 0.00345475] and parameters: {'model': 'model_dense_43.keras', 'l': 488, 'N': 302}. 


Trial parameters: {'l': 131, 'N': 35}


[I 2024-01-01 14:40:36,057] Trial 47 finished with values: [2.6590849210145855, 0.00034212000000000023] and parameters: {'model': 'model_dense_43.keras', 'l': 11, 'N': 1}. 
[I 2024-01-01 14:44:04,755] Trial 46 finished with values: [1.014067718379249, 0.004030540000000001] and parameters: {'model': 'model_dense_1.keras', 'l': 428, 'N': 37}. 
[I 2024-01-01 15:04:40,296] Trial 48 finished with values: [1.0471925973851037, 0.0023487699999999996] and parameters: {'model': 'model_dense_1.keras', 'l': 763, 'N': 343}. 
[I 2024-01-01 15:06:45,658] Trial 49 finished with values: [1.4875101062865317, 0.0005590599999999998] and parameters: {'model': 'model_dense_1.keras', 'l': 131, 'N': 35}. 


-------------------- Best trials --------------------
Trial no. 25
 Values = [1.0058222450393872, 0.005968559999999999]
 Params = {'model': 'model_dense_26.keras', 'l': 883, 'N': 140}
Trial no. 8
 Values = [1.007688726187458, 0.005568860000000001]
 Params = {'model': 'model_conv_46.keras', 'l': 859, 'N': 191}
Trial no. 29
 Values = [1.010682236832265, 0.002992389999999999]
 Params = {'model': 'model_conv_19.keras', 'l': 507, 'N': 73}
Trial no. 48
 Values = [1.0471925973851037, 0.0023487699999999996]
 Params = {'model': 'model_dense_1.keras', 'l': 763, 'N': 343}
Trial no. 23
 Values = [1.0671535552910947, 0.0019295099999999993]
 Params = {'model': 'model_dense_26.keras', 'l': 381, 'N': 125}
Trial no. 36
 Values = [1.2847339740216845, 0.0014960400000000003]
 Params = {'model': 'model_conv_12.keras', 'l': 273, 'N': 183}
Trial no. 39
 Values = [1.3241004328504322, 0.0013975300000000003]
 Params = {'model': 'model_conv_19.keras', 'l': 272, 'N': 199}
Trial no. 43
 Values = [1.341152122738061

In [40]:
df = mrng_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_N,params_l,params_model,system_attrs_nsga2:generation,state
0,25,1.005822,0.005969,2024-01-01 12:16:01.610080,2024-01-01 12:36:03.037141,0 days 00:20:01.427061,140,883,model_dense_26.keras,0,COMPLETE
1,8,1.007689,0.005569,2024-01-01 10:42:58.064815,2024-01-01 11:02:00.125343,0 days 00:19:02.060528,191,859,model_conv_46.keras,0,COMPLETE
2,29,1.010682,0.002992,2024-01-01 12:36:03.040827,2024-01-01 13:00:14.835784,0 days 00:24:11.794957,73,507,model_conv_19.keras,0,COMPLETE
3,35,1.011472,0.005787,2024-01-01 13:08:28.533679,2024-01-01 13:26:45.746401,0 days 00:18:17.212722,315,993,model_conv_46.keras,0,COMPLETE
4,26,1.012829,0.003265,2024-01-01 12:19:26.327467,2024-01-01 12:36:54.006562,0 days 00:17:27.679095,138,635,model_conv_46.keras,0,COMPLETE
5,21,1.013566,0.004316,2024-01-01 11:44:32.605349,2024-01-01 12:05:31.171458,0 days 00:20:58.566109,115,626,model_dense_26.keras,0,COMPLETE
6,46,1.014068,0.004031,2024-01-01 14:11:24.712974,2024-01-01 14:44:04.755363,0 days 00:32:40.042389,37,428,model_dense_1.keras,0,COMPLETE
7,19,1.025987,0.004847,2024-01-01 11:43:51.745753,2024-01-01 12:16:01.606078,0 days 00:32:09.860325,78,456,model_dense_1.keras,0,COMPLETE
8,6,1.026857,0.003466,2024-01-01 10:25:12.027539,2024-01-01 10:42:58.062941,0 days 00:17:46.035402,40,358,model_conv_46.keras,0,COMPLETE
9,15,1.026982,0.004152,2024-01-01 11:19:07.465206,2024-01-01 11:43:51.741832,0 days 00:24:44.276626,257,818,model_dense_43.keras,0,COMPLETE


## Visualize MRNG study results

In [41]:
plot_pareto_front(mrng_study, target_names=['aaf', 'average_time'])

In [42]:
plot_optimization_history(mrng_study, target = lambda t: t.values[0], target_name = 'aaf')

In [43]:
plot_optimization_history(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')

In [44]:
plot_slice(mrng_study, target = lambda t: t.values[0], target_name = 'aaf')

In [45]:
plot_slice(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')

# 5. Optimize NSG

To skip logs, click [here](#visualize-nsg-study-results).

In [46]:
def objective_nsg(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'m' : trial.suggest_int('m', 3, 200),
                  'l' : trial.suggest_int('l', 10, 1000),
                  'lq': trial.suggest_int('lq', 1, 1000),
                  'k' : trial.suggest_int('k', 40, 100)}
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]

    average_time, aaf_latent = nsg_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, N=5, int_data=0)

    return aaf_latent.value, average_time.value

In [47]:
%%time
nsg_study = optuna.create_study(study_name='nsg', directions=['minimize', 'minimize'])
nsg_study.optimize(objective_nsg, n_trials=100, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(nsg_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2024-01-01 15:06:45,910] A new study created in memory with name: nsg


Trial parameters: {'m': 99, 'l': 663, 'lq': 533, 'k': 41}
Trial parameters: {'m': 28, 'l': 91, 'lq': 430, 'k': 86}
Trial parameters: {'m': 76, 'l': 725, 'lq': 221, 'k': 100}
Trial parameters: {'m': 7, 'l': 39, 'lq': 127, 'k': 91}


[I 2024-01-01 15:10:22,777] Trial 2 finished with values: [1.0045123280983541, 0.00166804] and parameters: {'model': 'model_dense_26.keras', 'm': 76, 'l': 725, 'lq': 221, 'k': 100}. 


Trial parameters: {'m': 80, 'l': 336, 'lq': 884, 'k': 59}


[I 2024-01-01 15:12:00,182] Trial 0 finished with values: [1.022573229449533, 0.005184519999999999] and parameters: {'model': 'model_conv_12.keras', 'm': 99, 'l': 663, 'lq': 533, 'k': 41}. 


...

Trial parameters: {'m': 162, 'l': 870, 'lq': 768, 'k': 86}


[I 2024-01-01 19:47:31,944] Trial 89 finished with values: [1.2355107242779462, 0.012362319999999993] and parameters: {'model': 'model_conv_19.keras', 'm': 110, 'l': 234, 'lq': 977, 'k': 85}. 


Trial parameters: {'m': 99, 'l': 775, 'lq': 53, 'k': 93}


[I 2024-01-01 19:48:06,329] Trial 95 finished with values: [2.2307491241039026, 0.0008939000000000004] and parameters: {'model': 'model_conv_12.keras', 'm': 149, 'l': 557, 'lq': 82, 'k': 54}. 
[I 2024-01-01 19:50:08,857] Trial 98 finished with values: [1.0058279133164867, 0.004654339999999999] and parameters: {'model': 'model_dense_43.keras', 'm': 162, 'l': 870, 'lq': 768, 'k': 86}. 
[I 2024-01-01 19:50:26,221] Trial 99 finished with values: [1.7822358072951272, 0.00033258000000000007] and parameters: {'model': 'model_dense_1.keras', 'm': 99, 'l': 775, 'lq': 53, 'k': 93}. 
[I 2024-01-01 19:54:00,641] Trial 97 finished with values: [1.1514075629915954, 0.00123146] and parameters: {'model': 'model_conv_19.keras', 'm': 23, 'l': 612, 'lq': 301, 'k': 44}. 


-------------------- Best trials --------------------
Trial no. 94
 Values = [1.0, 0.004471589999999998]
 Params = {'model': 'model_dense_26.keras', 'm': 52, 'l': 987, 'lq': 667, 'k': 69}
Trial no. 41
 Values = [1.0000699015464658, 0.003309650000000001]
 Params = {'model': 'model_conv_46.keras', 'm': 43, 'l': 987, 'lq': 476, 'k': 87}
Trial no. 86
 Values = [1.0033123916516016, 0.0020207800000000007]
 Params = {'model': 'model_dense_26.keras', 'm': 119, 'l': 912, 'lq': 221, 'k': 48}
Trial no. 2
 Values = [1.0045123280983541, 0.00166804]
 Params = {'model': 'model_dense_26.keras', 'm': 76, 'l': 725, 'lq': 221, 'k': 100}
Trial no. 14
 Values = [1.0196882547858321, 0.00151628]
 Params = {'model': 'model_dense_43.keras', 'm': 192, 'l': 912, 'lq': 322, 'k': 49}
Trial no. 17
 Values = [1.0361899888151973, 0.0014078399999999996]
 Params = {'model': 'model_dense_1.keras', 'm': 183, 'l': 431, 'lq': 284, 'k': 53}
Trial no. 97
 Values = [1.1514075629915954, 0.00123146]
 Params = {'model': 'model_c

In [48]:
df = nsg_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_k,params_l,params_lq,params_m,params_model,system_attrs_nsga2:generation,state
0,94,1.000000,0.004472,2024-01-01 19:37:53.023307,2024-01-01 19:40:46.223003,0 days 00:02:53.199696,69,987,667,52,model_dense_26.keras,1,COMPLETE
1,41,1.000070,0.003310,2024-01-01 17:02:08.062536,2024-01-01 17:04:30.639045,0 days 00:02:22.576509,87,987,476,43,model_conv_46.keras,0,COMPLETE
2,73,1.000656,0.005145,2024-01-01 18:38:30.445519,2024-01-01 18:41:33.388547,0 days 00:03:02.943028,97,825,667,107,model_conv_46.keras,1,COMPLETE
3,38,1.000715,0.006091,2024-01-01 16:56:17.431508,2024-01-01 16:58:23.248717,0 days 00:02:05.817209,75,824,760,184,model_dense_26.keras,0,COMPLETE
4,34,1.002702,0.005953,2024-01-01 16:45:59.135134,2024-01-01 16:48:50.332011,0 days 00:02:51.196877,86,960,768,102,model_dense_43.keras,0,COMPLETE
5,72,1.003108,0.005171,2024-01-01 18:37:38.099395,2024-01-01 18:39:56.884430,0 days 00:02:18.785035,64,912,725,192,model_dense_26.keras,1,COMPLETE
6,86,1.003312,0.002021,2024-01-01 19:20:01.718932,2024-01-01 19:22:56.226631,0 days 00:02:54.507699,48,912,221,119,model_dense_26.keras,1,COMPLETE
7,2,1.004512,0.001668,2024-01-01 15:06:45.920258,2024-01-01 15:10:22.777206,0 days 00:03:36.856948,100,725,221,76,model_dense_26.keras,0,COMPLETE
8,87,1.004873,0.001736,2024-01-01 19:22:56.228318,2024-01-01 19:25:03.975729,0 days 00:02:07.747411,49,719,249,129,model_conv_46.keras,1,COMPLETE
9,7,1.005061,0.003853,2024-01-01 15:24:41.896271,2024-01-01 15:27:23.847822,0 days 00:02:41.951551,52,391,545,107,model_dense_26.keras,0,COMPLETE


## Visualize NSG study results

In [49]:
plot_pareto_front(nsg_study, target_names=['aaf', 'average_time'])

In [50]:
plot_optimization_history(nsg_study, target = lambda t: t.values[0], target_name = 'aaf')

In [51]:
plot_optimization_history(nsg_study, target = lambda t: t.values[1], target_name = 'average_time')

In [52]:
plot_slice(nsg_study, target = lambda t: t.values[0], target_name = 'aaf')

In [53]:
plot_slice(nsg_study, target = lambda t: t.values[1], target_name = 'average_time')

# 6. Grid Search

# 7. Results

## 7. a. Optimization Results

## 7. b. Grid Search Results

# 8. Conclusions